In [2]:
from io import StringIO
import glob
import os
import re
# the peak callers all inherit from DatabaseApi, a class which offers an interface 
# to a sqlite database to store Calling Cards data
from callingcardstools.database_managers.yeast import HopsDb
from callingcardstools.PackageResources import Resources
import pandas as pd

# This object allows retrieval of package resources
cc_resources = Resources()

# create a database either in memory or at a specified location
#yeast_db = hopsdb("/home/oguzkhan/Desktop/cc_metadata/hops_db.sqlite")
yeast_db = HopsDb("/home/oguzkhan/projects/rank_response_shiny/data/qc_db_v2.sqlite")

Checking table column names...
Current database tables are valid


In [3]:
from callingcardstools.BarcodeParser import BarcodeParser
# d = {
# 	'run_5301_5088': ['/mnt/scratch/calling_cards/sequence/run_5301_5088/run_5301_5088_barcode_details.json','/mnt/scratch/calling_cards/sequence/run_5301_5088/cctools_split/id_bc_map.tsv'],
# 	'run_5690': ['/mnt/scratch/calling_cards/sequence/run_5690/run_5690_barcode_details.json', '/mnt/scratch/calling_cards/sequence/run_5690/cctools_split/id_bc_map.tsv'],
# 	'run_6100': ['/mnt/scratch/calling_cards/sequence/run_6100/run_6100_barcode_details.json','/mnt/scratch/calling_cards/sequence/run_6100/cctools_split/id_bc_map.tsv'],
# 	'run_6106': ['/mnt/scratch/calling_cards/sequence/run_6106/run_6106_barcode_details.json','/mnt/scratch/calling_cards/sequence/run_6106/cctools_split/id_bc_map.tsv'],
# }
d = {
	'run_6354': ['/mnt/scratch/cc/mitra_pipeline/jp_test_run_full/run_6354_barcode_details.json',
	      '/mnt/scratch/cc/mitra_pipeline/jp_test_run_full/cctools_split/id_bc_map.tsv']
}

for k,v in d.items():
	bp = BarcodeParser(v[0])
	yeast_db.add_batch_qc(bp,v[1])

In [15]:
ccf_df = pd.DataFrame.from_dict(
	{'batch': ['run_6354', 'run_6354'],
     'tf': ['MTH1', 'SKN7'],
     'replicate': ['none', 'none'],
	 'ccf':['/mnt/scratch/cc/mitra_pipeline/jp_test_run_full/ccf/MTH1_with_annote.ccf', 
            '/mnt/scratch/cc/mitra_pipeline/jp_test_run_full/ccf/SKN7_with_annote.ccf']}
)

#ccf_df = pd.read_csv("/home/oguzkhan/projects/rank_response_shiny/data/run_6177_ccf_lookup.csv")
ccf_df['batch'] = ccf_df['batch'].str.lower()
batch_tbl = pd.read_sql_query('Select * from batch', yeast_db.con)
ccf_df_with_batch_id = pd.merge(ccf_df,batch_tbl,how='left', on=['batch','tf', 'replicate'])

In [20]:

df_list = []
def add_ccf_to_db(row: pd.Series) -> None:
	"""read in the ccf, augment and add to the yeast_db

	Args:
		row (pd.Series): a row from the ccf_df
	"""
	# note that only the first 6 rows are used for the names. sample, the 6th, 
	# is added in this function
	df = pd.read_csv(row['ccf'], 
	                 sep = '\t', 
					 names = ['chr','start','end','depth','strand','annotation'])
	df['batch_id'] = row['id']
	print(df.head(2))
	yeast_db.add_frame(df,'qbed',table_type='experiment',tablename_suffix = row['tf'], fk_tablelist=['batch'])



In [21]:
#ccf_df.apply(lambda row: extract_descriptors(row), axis=1, result_type='expand')
ccf_df_with_batch_id.apply(lambda row: add_ccf_to_db(row), axis=1)

    chr  start   end  depth strand     annotation  batch_id
0  chrI    176   177      2      +  GTCCCCAGAGGGG       114
1  chrI   2513  2514      1      -  GTCCCCAGAGGGG       114
    chr  start    end  depth strand     annotation  batch_id
0  chrI   6608   6609     14      -  TCAAGATCAGACC       115
1  chrI  15420  15421      1      +  TCAAGATCAGACC       115


0    None
1    None
dtype: object

In [23]:
#background_and_expr_tbls = [x for x in yeast_db.list_tables(yeast_db.con) if re.search(r"^background|^experiment",x)]
background_and_expr_tbls = ['experiment_SKN7', 'experiment_MTH1']
for regions_tbl in ['regions_yiming', 'regions_not_orf']:
    for qbed in background_and_expr_tbls:
        yeast_db.create_aggregate_view(qbed,regions_tbl)

In [25]:
# experiment_list = [x for x in yeast_db.list_tables(yeast_db.con) if re.match('^experiment_', x)]

experiment_list = ['experiment_SKN7', 'experiment_MTH1'] 

for region_tbl in ['regions_yiming', 'regions_not_orf']:
	print(f"region: {region_tbl}")
	for background_tbl in ['background_adh1', 'background_dSir4']:
		print(f"background: {background_tbl}")
		for experiment_tbl in experiment_list:
			#print(f"experiment: {experiment_tbl}")
			yeast_db.peak_caller(regions = region_tbl,background = background_tbl, experiment = experiment_tbl,if_exists='replace')

region: regions_yiming
background: background_adh1
background: background_dSir4
region: regions_not_orf
background: background_adh1
background: background_dSir4
